In [ ]:
import sys
import os

notebook_path = os.getcwd() 
parent_dir = os.path.dirname(notebook_path)
project_root = os.path.dirname(parent_dir)
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
from copy import deepcopy

In [ ]:
import state_NN_models
import Filters
import utils
import Systems
from utils import losses, trainer, utils
from torch.utils.data import TensorDataset, DataLoader, random_split
from state_NN_models.StateBayesianKalmanNet import StateBayesianKalmanNet
from state_NN_models.StateKalmanNet import StateKalmanNet
from state_NN_models.StateKalmanNetWithKnownR import StateKalmanNetWithKnownR

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Používané zařízení: {device}")

In [ ]:
state_dim = 1
obs_dim = 1
h_true_nonlinear = lambda x: 0.5 * x
f_true_nonlinear = lambda x: 0.9 * x - 0.05 * x**3 

Q_true = torch.tensor([[0.1]])
R_true = torch.tensor([[0.1]])

Ex0_true = torch.tensor([[1.0]])
P0_true = torch.tensor([[0.5]])

sys_true = Systems.DynamicSystem(state_dim=state_dim, obs_dim=obs_dim, f=f_true_nonlinear, h=h_true_nonlinear, Q=Q_true, R=R_true, Ex0=Ex0_true, P0=P0_true,device=device)

#  Nepřesná dynamika (lineární aproximace nelineární funkce f)
f_model_nonlinear = lambda x: 0.9 * x 
h_model_nonlinear = h_true_nonlinear
# Nepřesná znalost šumu (podcenění Q)
Q_model = torch.tensor([[0.01]])
R_model = torch.tensor([[0.2]])
# Nepřesný počáteční odhad (pro EKF)
Ex0_model = torch.tensor([[0.5]])
P0_model = torch.tensor([[0.5]])

# Sestavení nepřesného modelu pro filtry
# Funkce h, R jsou pro jednoduchost stejné, ale f, Q, Ex0, P0 jsou jiné
# sys_model = NonlinearSystem(f_model_nonlinear, h_model_nonlinear, Q_model, R_model, Ex0_model, P0_model)
sys_model = Systems.DynamicSystem(state_dim=state_dim, obs_dim=obs_dim, f=f_model_nonlinear, h=h_model_nonlinear, Q=Q_model, R=R_model, Ex0=Ex0_model, P0=P0_model,device=device)

In [6]:
import torch
import torch.nn.functional as F
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

# ==============================================================================
# 1. KONFIGURACE TESTU
# ==============================================================================
TEST_SEQ_LEN = 300
NUM_TEST_TRAJ = 2
J_SAMPLES_TEST = 25

# ==============================================================================
# 2. PŘÍPRAVA DAT
# ==============================================================================
print(f"\nGeneruji {NUM_TEST_TRAJ} testovacích trajektorií o délce {TEST_SEQ_LEN}...")
x_test, y_test = utils.generate_data(sys_true, num_trajectories=NUM_TEST_TRAJ, seq_len=TEST_SEQ_LEN)
test_dataset = TensorDataset(x_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
print("Generování dat dokončeno.")


# ==============================================================================
# 3. INICIALIZACE VŠECH FILTRŮ PRO POROVNÁNÍ
# ==============================================================================
ekf_mismatched = Filters.ExtendedKalmanFilter(sys_model)
ekf_ideal = Filters.ExtendedKalmanFilter(sys_true)
ukf_mismatched = Filters.UnscentedKalmanFilter(sys_model)
ukf_ideal = Filters.UnscentedKalmanFilter(sys_true)
aekf_mismatched = Filters.AdaptiveExtendedKalmanFilter(sys_model, Q_init=sys_model.Q, R_init=sys_model.R, alpha=0.985)
# kf_ideal = Filters.KalmanFilter(sys_true)

# akf_mismatched = Filters.AdaptiveKalmanFilter(sys_model,mdm_L=3,mdm_version=2)
# akf_ideal = Filters.AdaptiveKalmanFilter(sys_true,mdm_L=3,mdm_version=2)

# pf_ideal = Filters.ParticleFilter(sys_true, num_particles=1000)
# pf_mismatched = Filters.ParticleFilter(sys_model_known_init, num_particles=1000)

pf_sir_ideal = Filters.ParticleFilterSIR(sys_true, num_particles=1000)
pf_sir_mismatched = Filters.ParticleFilterSIR(sys_model, num_particles=1000)

apf_ideal = Filters.AuxiliaryParticleFilter(sys_true, num_particles=1000)
apf_mismatched = Filters.AuxiliaryParticleFilter(sys_model, num_particles=1000)

pf_mh_ideal = Filters.ParticleFilterMH(sys_true, num_particles=1000)
pf_mh_mismatched = Filters.ParticleFilterMH(sys_model, num_particles=1000)

print("Všechny model-based filtry (EKF, UKF, AEKF) inicializovány.")


# ==============================================================================
# 4. VYHODNOCOVACÍ SMYČKA
# ==============================================================================
# Seznamy pro ukládání výsledků z každé trajektorie
all_x_true_cpu = []
all_x_hat_bkn_cpu, all_P_hat_bkn_cpu = [], []
all_x_hat_classic_knet_cpu = []
all_x_hat_knetR_cpu, all_P_hat_knetR_cpu = [], []
all_x_hat_ekf_mismatched_cpu, all_P_hat_ekf_mismatched_cpu = [], []
all_x_hat_ekf_ideal_cpu, all_P_hat_ekf_ideal_cpu = [], []
all_x_hat_ukf_mismatched_cpu, all_P_hat_ukf_mismatched_cpu = [], []
all_x_hat_ukf_ideal_cpu, all_P_hat_ukf_ideal_cpu = [], []
all_x_hat_aekf_mismatched_cpu, all_P_hat_aekf_mismatched_cpu = [], []
all_x_hat_kf_ideal_cpu, all_P_hat_kf_ideal_cpu = [], []
all_x_hat_pf_ideal_cpu, all_P_hat_pf_ideal_cpu = [], []
all_x_hat_pf_mismatched_cpu, all_P_hat_pf_mismatched_cpu = [], []
all_x_hat_pf_sir_ideal_cpu, all_P_hat_pf_sir_ideal_cpu = [], []
all_x_hat_pf_sir_mismatched_cpu, all_P_hat_pf_sir_mismatched_cpu = [], []
all_x_hat_apf_ideal_cpu, all_P_hat_apf_ideal_cpu = [], []
all_x_hat_apf_mismatched_cpu, all_P_hat_apf_mismatched_cpu = [], []
all_x_hat_pf_mh_ideal_cpu, all_P_hat_pf_mh_ideal_cpu = [], []
all_x_hat_pf_mh_mismatched_cpu, all_P_hat_pf_mh_mismatched_cpu = [], []
all_x_hat_akf_ideal_cpu, all_P_hat_akf_ideal_cpu = [], []
all_x_hat_akf_mismatched_cpu, all_P_hat_akf_mismatched_cpu = [], []


print(f"\nVyhodnocuji modely na {NUM_TEST_TRAJ} testovacích trajektoriích...")


with torch.no_grad():
    for i, (x_true_seq_batch, y_test_seq_batch) in enumerate(test_loader):
        y_test_seq_gpu = y_test_seq_batch.squeeze(0).to(device)
        x_true_seq_gpu = x_true_seq_batch.squeeze(0).to(device)
        initial_state = x_true_seq_gpu[0, :].unsqueeze(0)
        
      # --- D. EKF (nepřesný a ideální) ---
        ekf_m_res = ekf_mismatched.process_sequence(y_test_seq_gpu, Ex0=sys_model.Ex0, P0=sys_model.P0)

        full_x_hat_ekf_m = ekf_m_res['x_filtered'] # Výsledek je již kompletní trajektorie
        full_P_hat_ekf_m = ekf_m_res['P_filtered'] # To samé pro kovarianci

        ekf_i_res = ekf_ideal.process_sequence(y_test_seq_gpu, Ex0=sys_true.Ex0, P0=sys_true.P0)

        full_x_hat_ekf_i = ekf_i_res['x_filtered']
        full_P_hat_ekf_i = ekf_i_res['P_filtered']

        # --- E. UKF (nepřesný a ideální) ---
        ukf_m_res = ukf_mismatched.process_sequence(y_test_seq_gpu, Ex0=sys_model.Ex0, P0=sys_model.P0)

        full_x_hat_ukf_m = ukf_m_res['x_filtered']
        full_P_hat_ukf_m = ukf_m_res['P_filtered']

        ukf_i_res = ukf_ideal.process_sequence(y_test_seq_gpu, Ex0=sys_true.Ex0, P0=sys_true.P0)

        full_x_hat_ukf_i = ukf_i_res['x_filtered']
        full_P_hat_ukf_i = ukf_i_res['P_filtered']

        # kf_i_res = kf_ideal.process_sequence(y_test_seq_gpu, Ex0=sys_true.Ex0, P0=sys_true.P0)

        # full_x_hat_kf_i = kf_i_res['x_filtered']
        # full_P_hat_kf_i = kf_i_res['P_filtered']



        apf_i_res = apf_ideal.process_sequence(y_test_seq_gpu, Ex0=sys_true.Ex0,P0=sys_true.P0)
        full_x_hat_apf_i = apf_i_res['x_filtered']
        full_P_hat_apf_i = apf_i_res['P_filtered']
        print(f"APF (ideální model) dokončen pro trajektorii {i + 1}/{NUM_TEST_TRAJ}.")

        apf_m_res = apf_mismatched.process_sequence(y_test_seq_gpu, Ex0=sys_model.Ex0, P0=sys_model.P0)
        full_x_hat_apf_m = apf_m_res['x_filtered']
        full_P_hat_apf_m = apf_m_res['P_filtered']
        print(f"APF (nepřesný model) dokončen pro trajektorii {i + 1}/{NUM_TEST_TRAJ}.")



        pf_sir_i_res = pf_sir_ideal.process_sequence(y_test_seq_gpu, Ex0=sys_true.Ex0,P0=sys_true.P0)
        full_x_hat_pf_sir_i = pf_sir_i_res['x_filtered']
        full_P_hat_pf_sir_i = pf_sir_i_res['P_filtered']
        full_particles_history_pf_sir_i = pf_sir_i_res['particles_history']
        print(f"PF-SIR (ideální model) dokončen pro trajektorii {i + 1}/{NUM_TEST_TRAJ}.")

        pf_sir_m_res = pf_sir_mismatched.process_sequence(y_test_seq_gpu, Ex0=sys_model.Ex0, P0=sys_model.P0)
        full_x_hat_pf_sir_m = pf_sir_m_res['x_filtered']
        full_P_hat_pf_sir_m = pf_sir_m_res['P_filtered']
        full_particles_history_pf_sir_m = pf_sir_m_res['particles_history']
        print(f"PF-SIR (nepřesný model) dokončen pro trajektorii {i + 1}/{NUM_TEST_TRAJ}.")

        pf_mh_i_res = pf_mh_ideal.process_sequence(y_test_seq_gpu, Ex0=sys_true.Ex0,P0=sys_true.P0)
        full_x_hat_pf_mh_i = pf_mh_i_res['x_filtered']
        full_P_hat_pf_mh_i = pf_mh_i_res['P_filtered']
        print(f"PF-MH (ideální model) dokončen pro trajektorii {i + 1}/{NUM_TEST_TRAJ}.")

        pf_mh_m_res = pf_mh_mismatched.process_sequence(y_test_seq_gpu, Ex0=sys_model.Ex0, P0=sys_model.P0)
        full_x_hat_pf_mh_m = pf_mh_m_res['x_filtered']
        full_P_hat_pf_mh_m = pf_mh_m_res['P_filtered']
        print(f"PF-MH (nepřesný model) dokončen pro trajektorii {i + 1}/{NUM_TEST_TRAJ}.")

        # akf_i_res,_,_ = akf_ideal.process_sequence_adaptively(y_test_seq_gpu)
        # full_x_hat_akf_i = akf_i_res['x_filtered']
        # full_P_hat_akf_i = akf_i_res['P_filtered']

        # akf_m_res,_,_ = akf_mismatched.process_sequence_adaptively(y_test_seq_gpu)
        # full_x_hat_akf_m = akf_m_res['x_filtered']
        # full_P_hat_akf_m = akf_m_res['P_filtered']

        # --- G. Uložení všech výsledků na CPU ---
        all_x_true_cpu.append(x_true_seq_gpu.cpu())
        all_x_hat_ekf_mismatched_cpu.append(full_x_hat_ekf_m.cpu()); all_P_hat_ekf_mismatched_cpu.append(full_P_hat_ekf_m.cpu())
        all_x_hat_ekf_ideal_cpu.append(full_x_hat_ekf_i.cpu()); all_P_hat_ekf_ideal_cpu.append(full_P_hat_ekf_i.cpu())
        all_x_hat_ukf_mismatched_cpu.append(full_x_hat_ukf_m.cpu()); all_P_hat_ukf_mismatched_cpu.append(full_P_hat_ukf_m.cpu())
        all_x_hat_ukf_ideal_cpu.append(full_x_hat_ukf_i.cpu()); all_P_hat_ukf_ideal_cpu.append(full_P_hat_ukf_i.cpu())
        # all_x_hat_kf_ideal_cpu.append(full_x_hat_kf_i.cpu()); all_P_hat_kf_ideal_cpu.append(full_P_hat_kf_i.cpu())
        # all_x_hat_pf_ideal_cpu.append(full_x_hat_pf_i.cpu()); all_P_hat_pf_ideal_cpu.append(full_P_hat_pf_i.cpu())
        # all_x_hat_pf_mismatched_cpu.append(full_x_hat_pf_m.cpu()); all_P_hat_pf_mismatched_cpu.append(full_P_hat_pf_m.cpu())
        all_x_hat_pf_sir_ideal_cpu.append(full_x_hat_pf_sir_i.cpu()); all_P_hat_pf_sir_ideal_cpu.append(full_P_hat_pf_sir_i.cpu())
        all_x_hat_pf_sir_mismatched_cpu.append(full_x_hat_pf_sir_m.cpu()); all_P_hat_pf_sir_mismatched_cpu.append(full_P_hat_pf_sir_m.cpu())
        all_x_hat_apf_ideal_cpu.append(full_x_hat_apf_i.cpu()); all_P_hat_apf_ideal_cpu.append(full_P_hat_apf_i.cpu())
        all_x_hat_apf_mismatched_cpu.append(full_x_hat_apf_m.cpu()); all_P_hat_apf_mismatched_cpu.append(full_P_hat_apf_m.cpu())
        all_x_hat_pf_mh_ideal_cpu.append(full_x_hat_pf_mh_i.cpu()); all_P_hat_pf_mh_ideal_cpu.append(full_P_hat_pf_mh_i.cpu())
        all_x_hat_pf_mh_mismatched_cpu.append(full_x_hat_pf_mh_m.cpu()); all_P_hat_pf_mh_mismatched_cpu.append(full_P_hat_pf_mh_m.cpu())
        # all_x_hat_akf_ideal_cpu.append(full_x_hat_akf_i.cpu()); all_P_hat_akf_ideal_cpu.append(full_P_hat_akf_i.cpu())
        # all_x_hat_akf_mismatched_cpu.append(full_x_hat_akf_m.cpu()); all_P_hat_akf_mismatched_cpu.append(full_P_hat_akf_m.cpu())

        print(f"Dokončena trajektorie {i + 1}/{NUM_TEST_TRAJ}...")

# ==============================================================================
# 5. FINÁLNÍ VÝPOČET A VÝPIS METRIK
# ==============================================================================
# Seznamy pro sběr metrik
mse_bkn, anees_bkn = [], []; mse_classic_knet = []; mse_knetR, anees_knetR = [], []
mse_ekf_mis, anees_ekf_mis = [], []; mse_ekf_ideal, anees_ekf_ideal = [], []
mse_ukf_mis, anees_ukf_mis = [], []; mse_ukf_ideal, anees_ukf_ideal = [], []
mse_aekf_mis, anees_aekf_mis = [], []; mse_kf_ideal, anees_kf_ideal = [], []
mse_pf_ideal, anees_pf_ideal = [], []; mse_pf_mis, anees_pf_mis = [], []
mse_pf_sir_ideal, anees_pf_sir_ideal = [], []; mse_pf_sir_mis, anees_pf_sir_mis = [], []
mse_apf_ideal, anees_apf_ideal = [], []; mse_apf_mis, anees_apf_mis = [], []
all_particles_history_pf_sir_mismatched_cpu = []
all_particles_history_pf_sir_ideal_cpu = []
mse_pf_mh_ideal, anees_pf_mh_ideal = [], []; mse_pf_mh_mis, anees_pf_mh_mis = [], []
mse_akf_ideal, anees_akf_ideal = [], []; mse_akf_mis, anees_akf_mis = [], []

print("\nPočítám finální metriky pro jednotlivé trajektorie...")

with torch.no_grad():
    for i in range(NUM_TEST_TRAJ):
        x_true = all_x_true_cpu[i]
        def get_metrics(x_hat, P_hat):
            # Délka pravdivé trajektorie
            true_len = x_true.shape[0]
            
            # Ořízneme odhady, aby měly stejnou délku jako pravdivá data.
            # Bereme prvních `true_len` prvků (tj. pro t=0 až t=299).
            x_hat_aligned = x_hat[:true_len]
            P_hat_aligned = P_hat[:true_len]
            
            # Nyní mají všechny tenzory shodnou délku
            mse = F.mse_loss(x_hat_aligned, x_true).item()
            anees = utils.calculate_anees_vectorized(x_true.unsqueeze(0), x_hat_aligned.unsqueeze(0), P_hat_aligned.unsqueeze(0))
            return mse, anees

        # Výpočty pro všechny modely
        mse, anees = get_metrics(all_x_hat_ekf_mismatched_cpu[i], all_P_hat_ekf_mismatched_cpu[i]); mse_ekf_mis.append(mse); anees_ekf_mis.append(anees)
        mse, anees = get_metrics(all_x_hat_ekf_ideal_cpu[i], all_P_hat_ekf_ideal_cpu[i]); mse_ekf_ideal.append(mse); anees_ekf_ideal.append(anees)
        mse, anees = get_metrics(all_x_hat_ukf_mismatched_cpu[i], all_P_hat_ukf_mismatched_cpu[i]); mse_ukf_mis.append(mse); anees_ukf_mis.append(anees)
        mse, anees = get_metrics(all_x_hat_ukf_ideal_cpu[i], all_P_hat_ukf_ideal_cpu[i]); mse_ukf_ideal.append(mse); anees_ukf_ideal.append(anees)
        # mse, anees = get_metrics(all_x_hat_kf_ideal_cpu[i], all_P_hat_kf_ideal_cpu[i]); mse_kf_ideal.append(mse); anees_kf_ideal.append(anees)
        # mse, anees = get_metrics(all_x_hat_pf_ideal_cpu[i], all_P_hat_pf_ideal_cpu[i]); mse_pf_ideal.append(mse); anees_pf_ideal.append(anees)
        # mse, anees = get_metrics(all_x_hat_pf_mismatched_cpu[i], all_P_hat_pf_mismatched_cpu[i]); mse_pf_mis.append(mse); anees_pf_mis.append(anees)
        mse, anees = get_metrics(all_x_hat_pf_sir_ideal_cpu[i], all_P_hat_pf_sir_ideal_cpu[i]); mse_pf_sir_ideal.append(mse); anees_pf_sir_ideal.append(anees)
        mse, anees = get_metrics(all_x_hat_pf_sir_mismatched_cpu[i], all_P_hat_pf_sir_mismatched_cpu[i]); mse_pf_sir_mis.append(mse); anees_pf_sir_mis.append(anees)
        mse, anees = get_metrics(all_x_hat_apf_ideal_cpu[i], all_P_hat_apf_ideal_cpu[i]); mse_apf_ideal.append(mse); anees_apf_ideal.append(anees)
        mse, anees = get_metrics(all_x_hat_apf_mismatched_cpu[i], all_P_hat_apf_mismatched_cpu[i]); mse_apf_mis.append(mse); anees_apf_mis.append(anees)
        mse, anees = get_metrics(all_x_hat_pf_mh_ideal_cpu[i], all_P_hat_pf_mh_ideal_cpu[i]); mse_pf_mh_ideal.append(mse); anees_pf_mh_ideal.append(anees)
        mse, anees = get_metrics(all_x_hat_pf_mh_mismatched_cpu[i], all_P_hat_pf_mh_mismatched_cpu[i]); mse_pf_mh_mis.append(mse); anees_pf_mh_mis.append(anees)
        # mse, anees = get_metrics(all_x_hat_akf_ideal_cpu[i], all_P_hat_akf_ideal_cpu[i]); mse_akf_ideal.append(mse); anees_akf_ideal.append(anees)
        # mse, anees = get_metrics(all_x_hat_akf_mismatched_cpu[i], all_P_hat_akf_mismatched_cpu[i]); mse_akf_mis.append(mse); anees_akf_mis.append(anees)

# Funkce pro bezpečné průměrování
def avg(metric_list): return np.mean([m for m in metric_list if not np.isnan(m)])
state_dim_for_nees = all_x_true_cpu[0].shape[1]

# --- Finální výpis tabulky ---
print("\n" + "="*80)
print(f"FINÁLNÍ VÝSLEDKY (průměr přes {NUM_TEST_TRAJ} běhů)")
print("="*80)
print(f"{'Model':<35} | {'Průměrné MSE':<20} | {'Průměrný ANEES':<20}")
print("-" * 80)
print("-" * 80)
print(f"{'--- Model-Based Filters ---':<35} | {'':<20} | {'':<20}")
print(f"{'EKF (Nepřesný model)':<35} | {avg(mse_ekf_mis):<20.4f} | {avg(anees_ekf_mis):<20.4f}")
print(f"{'UKF (Nepřesný model)':<35} | {avg(mse_ukf_mis):<20.4f} | {avg(anees_ukf_mis):<20.4f}")
# print(f"{'PF (Nepřesný model)':<35} | {avg(mse_pf_mis):<20.4f} | {avg(anees_pf_mis):<20.4f}")
print(f"{'PF-SIR (Nepřesný model)':<35} | {avg(mse_pf_sir_mis):<20.4f} | {avg(anees_pf_sir_mis):<20.4f}")
print(f"{'APF (Nepřesný model)':<35} | {avg(mse_apf_mis):<20.4f} | {avg(anees_apf_mis):<20.4f}")
print(f"{'PF-MH (Nepřesný model)':<35} | {avg(mse_pf_mh_mis):<20.4f} | {avg(anees_pf_mh_mis):<20.4f}")
# print(f"{'AKF (Nepřesný model)':<35} | {avg(mse_akf_mis):<20.4f} | {avg(anees_akf_mis):<20.4f}")
print("-" * 80)
print(f"{'--- Benchmarks ---':<35} | {'':<20} | {'':<20}")
print(f"{'EKF (Ideální model)':<35} | {avg(mse_ekf_ideal):<20.4f} | {avg(anees_ekf_ideal):<20.4f}")
print(f"{'UKF (Ideální model)':<35} | {avg(mse_ukf_ideal):<20.4f} | {avg(anees_ukf_ideal):<20.4f}")
# print(f"{'KF (Ideální model)':<35} | {avg(mse_kf_ideal):<20.4f} | {avg(anees_kf_ideal):<20.4f}")
# print(f"{'PF (Ideální model)':<35} | {avg(mse_pf_ideal):<20.4f} | {avg(anees_pf_ideal):<20.4f}")
print(f"{'PF-SIR (Ideální model)':<35} | {avg(mse_pf_sir_ideal):<20.4f} | {avg(anees_pf_sir_ideal):<20.4f}")
print(f"{'APF (Ideální model)':<35} | {avg(mse_apf_ideal):<20.4f} | {avg(anees_apf_ideal):<20.4f}")
print(f"{'PF-MH (Ideální model)':<35} | {avg(mse_pf_mh_ideal):<20.4f} | {avg(anees_pf_mh_ideal):<20.4f}")
# print(f"{'AKF (Ideální model)':<35} | {avg(mse_akf_ideal):<20.4f} | {avg(anees_akf_ideal):<20.4f}")
print("="*80)

APF (ideální model) dokončen pro trajektorii 1/2.
APF (nepřesný model) dokončen pro trajektorii 1/2.
PF-SIR (ideální model) dokončen pro trajektorii 1/2.
PF-SIR (nepřesný model) dokončen pro trajektorii 1/2.
PF-MH (ideální model) dokončen pro trajektorii 1/2.
PF-MH (nepřesný model) dokončen pro trajektorii 1/2.
Dokončena trajektorie 1/2...
APF (ideální model) dokončen pro trajektorii 2/2.
APF (nepřesný model) dokončen pro trajektorii 2/2.
PF-SIR (ideální model) dokončen pro trajektorii 2/2.
PF-SIR (nepřesný model) dokončen pro trajektorii 2/2.
PF-MH (ideální model) dokončen pro trajektorii 2/2.
PF-MH (nepřesný model) dokončen pro trajektorii 2/2.
Dokončena trajektorie 2/2...

Počítám finální metriky pro jednotlivé trajektorie...

FINÁLNÍ VÝSLEDKY (průměr přes 2 běhů)
Model                               | Průměrné MSE         | Průměrný ANEES      
--------------------------------------------------------------------------------
-----------------------------------------------------------

In [7]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# --- Importy pro interaktivitu ---
import ipywidgets as widgets
from ipywidgets import interact, IntSlider
from IPython.display import display

# ==============================================================================
# 1. KONFIGURACE A INICIALIZACE
# ==============================================================================
print("Inicializuji PF-SIR filtry pro diagnostiku...")

# Pro diagnostiku budeme pracovat jen s jednou trajektorií.
# Ujistěte se, že NUM_TEST_TRAJ=1 v buňce, kde generujete data.
if 'NUM_TEST_TRAJ' not in locals() or NUM_TEST_TRAJ > 1:
    if 'NUM_TEST_TRAJ' in locals():
        print(f"VAROVÁNÍ: NUM_TEST_TRAJ bylo {NUM_TEST_TRAJ}. Pro účely této diagnostiky se používá pouze první trajektorie.")
    else:
        # Pokud NUM_TEST_TRAJ neexistuje, nastavíme ho
        NUM_TEST_TRAJ = 1

# Inicializace filtrů, které chceme testovat
# Můžete si upravit počet částic pro experimentování
NUM_PARTICLES_DIAG = 200
pf_sir_ideal = Filters.ParticleFilterSIR(sys_true, num_particles=NUM_PARTICLES_DIAG)
pf_sir_mismatched = Filters.ParticleFilterSIR(sys_model_known_init, num_particles=NUM_PARTICLES_DIAG)

print("Inicializace dokončena.")

# ==============================================================================
# 2. ZPRACOVÁNÍ JEDNÉ TRAJEKTORIE
# ==============================================================================
print("\nZpracovávám první testovací trajektorii...")

# Vezmeme si první trajektorii z našich testovacích dat
x_true_seq_gpu = x_test[0].to(device)
y_test_seq_gpu = y_test[0].to(device)

with torch.no_grad():
    # --- BĚH IDEÁLNÍHO MODELU ---
    # Ujistěte se, že váš process_sequence vrací 'particles_history'
    pf_sir_i_res = pf_sir_ideal.process_sequence(
        y_test_seq_gpu, 
        Ex0=sys_true.Ex0,
        P0=sys_true.P0,
        Q=sys_true.Q,
        R=sys_true.R
    )
    print("PF-SIR (ideální model) dokončen.")

    # --- BĚH NEPŘESNÉHO MODELU ---
    pf_sir_m_res = pf_sir_mismatched.process_sequence(
        y_test_seq_gpu,
        Ex0=sys_model_known_init.Ex0,
        P0=sys_model_known_init.P0,
        Q=sys_model_known_init.Q,
        R=sys_model_known_init.R
    )
    print("PF-SIR (nepřesný model) dokončen.")

print("Zpracování trajektorie dokončeno.")

# ==============================================================================
# 3. INTERAKTIVNÍ VIZUALIZAČNÍ FUNKCE
# ==============================================================================

def plot_particle_filter_step(x_true, y_meas, pf_results, t, title_prefix=""):
    """
    Vykreslí stav filtru v JEDNOM konkrétním časovém kroku 't'.
    """
    # Převedeme data na NumPy jen jednou na začátku, pokud ještě nejsou
    x_true_np = x_true if isinstance(x_true, np.ndarray) else x_true.cpu().numpy()
    y_meas_np = y_meas if isinstance(y_meas, np.ndarray) else y_meas.cpu().numpy()
    x_est_np = pf_results['x_filtered'].cpu().numpy()
    particles_history = pf_results.get('particles_history')
    
    # Zjistíme dimenzi stavu a indexy pro osy
    state_dim = x_true_np.shape[1]
    x_axis_idx = 0
    y_axis_idx = 1
    if state_dim > 2 and state_dim % 2 == 0:
        y_axis_idx = state_dim // 2
        
    plt.figure(figsize=(11, 9))
    
    # Vykreslíme CELOU historii trajektorií a měření pro kontext
    plt.plot(x_true_np[:, x_axis_idx], x_true_np[:, y_axis_idx], 'g-', linewidth=2, alpha=0.3, label='Skutečná trajektorie (celá)')
    plt.scatter(y_meas_np[:, 0], y_meas_np[:, 1], color='b', marker='x', s=15, alpha=0.3, label='Měření (všechna)')
    plt.plot(x_est_np[:, x_axis_idx], x_est_np[:, y_axis_idx], 'r--', linewidth=1.5, alpha=0.3, label='Odhad filtru (celý)')

    # Zvýrazníme AKTUÁLNÍ pozice v čase 't'
    plt.plot(x_true_np[t, x_axis_idx], x_true_np[t, y_axis_idx], 'go', markersize=12, label=f'Skutečnost v t={t}')
    if t < len(y_meas_np):
        plt.plot(y_meas_np[t, 0], y_meas_np[t, 1], 'bo', markersize=12, markerfacecolor='none', markeredgewidth=2, label=f'Měření v t={t}')
    plt.plot(x_est_np[t, x_axis_idx], x_est_np[t, y_axis_idx], 'rs', markersize=8, label=f'Odhad v t={t}')
    
    # Vykreslíme AKTUÁLNÍ mrak částic
    if particles_history and t < len(particles_history):
        particles = particles_history[t]
        sample_size = min(len(particles), 1000)
        sample_indices = np.random.choice(len(particles), sample_size, replace=False)
        
        plt.scatter(particles[sample_indices, x_axis_idx], particles[sample_indices, y_axis_idx], 
                    color='black', s=10, alpha=0.6, label=f'Částice v t={t}')

    plt.xlabel(f"Stav [{x_axis_idx}]")
    plt.ylabel(f"Stav [{y_axis_idx}]")
    plt.title(f"{title_prefix} - Stav v čase t={t}", fontsize=16)
    plt.legend()
    plt.grid(True)
    plt.axis('equal')
    plt.show()

# ==============================================================================
# 4. SPUŠTĚNÍ INTERAKTIVNÍ VIZUALIZACE
# ==============================================================================
print("\nVytvářím interaktivní diagnostické grafy...")
print("Použijte slidery níže pro prozkoumání vývoje filtrů v čase.")

# Připravíme data pro předání funkcím (jen pro přehlednost)
x_true_np_diag = x_true_seq_gpu.cpu().numpy()
y_meas_np_diag = y_test_seq_gpu.cpu().numpy()

# Funkce, která bude volána sliderem pro IDEÁLNÍ model
def interactive_plot_ideal(t):
    # Zkontrolujeme, zda historie existuje
    if pf_sir_i_res.get('particles_history') is None:
        print("CHYBA: 'particles_history' nenalezena ve výsledcích ideálního filtru. Upravte metodu process_sequence, aby ji vracela.")
        return
    plot_particle_filter_step(x_true_np_diag, y_meas_np_diag, pf_sir_i_res, t, 
                              title_prefix="PF-SIR (Ideální model)")

# Funkce, která bude volána sliderem pro NEPŘESNÝ model
def interactive_plot_mismatched(t):
    # Zkontrolujeme, zda historie existuje
    if pf_sir_m_res.get('particles_history') is None:
        print("CHYBA: 'particles_history' nenalezena ve výsledcích nepřesného filtru. Upravte metodu process_sequence, aby ji vracela.")
        return
    plot_particle_filter_step(x_true_np_diag, y_meas_np_diag, pf_sir_m_res, t, 
                                title_prefix="PF-SIR (Nepřesný model)")

# Vytvoříme slidery a propojíme je s funkcemi
seq_len = x_true_np_diag.shape[0]

print("\n--- Interaktivní graf pro IDEÁLNÍ model ---")
interact(interactive_plot_ideal, t=IntSlider(min=0, max=seq_len-1, step=1, value=0, description="Časový krok (t):", continuous_update=False))

print("\n--- Interaktivní graf pro NEPŘESNÝ model ---")
interact(interactive_plot_mismatched, t=IntSlider(min=0, max=seq_len-1, step=1, value=0, description="Časový krok (t):", continuous_update=False))

Inicializuji PF-SIR filtry pro diagnostiku...
VAROVÁNÍ: NUM_TEST_TRAJ bylo 2. Pro účely této diagnostiky se používá pouze první trajektorie.


NameError: name 'sys_model_known_init' is not defined

In [ ]:
Q_history = aekf_m_res['Q_history']
R_history = aekf_m_res['R_history']

plt.figure(figsize=(20, 10))
plt.subplot(2, 1, 1)
plt.plot(Q_history[:,0,0].cpu(), label='Q[0,0]')
plt.plot(R_history[:,0,0].cpu(), label='R[0,0]')
plt.axhline(sys_true.Q[0,0].cpu().item(), color='blue', linestyle='--', label='True Q[0,0]')
plt.axhline(sys_true.R[0,0].cpu().item(), color='orange', linestyle='--', label='True R[0,0]')
plt.axhline(sys_model.Q[0,0].cpu().item(), color='blue', linestyle=':', label='Initial Q[0,0]')
plt.axhline(sys_model.R[0,0].cpu().item(), color='orange', linestyle=':', label='Initial R[0,0]')
plt.xlabel('Time Step')
plt.ylabel('Value')
plt.title('Adaptation of Q and R over Time')
plt.legend()
plt.grid()
plt.show()

plt.figure(figsize=(20, 10))
plt.subplot(2, 1, 2)
plt.plot(Q_history[:,1,1].cpu(), label='Q[1,1]')
plt.plot(R_history[:,1,1].cpu(), label='R[1,1]')
plt.axhline(sys_true.Q[1,1].cpu().item(), color='blue', linestyle='--', label='True Q[1,1]')
plt.axhline(sys_true.R[1,1].cpu().item(), color='orange', linestyle='--', label='True R[1,1]')
plt.axhline(sys_model.Q[1,1].cpu().item(), color='blue', linestyle=':', label='Initial Q[1,1]')
plt.axhline(sys_model.R[1,1].cpu().item(), color='orange', linestyle=':', label='Initial R[1,1]')
plt.xlabel('Time Step')
plt.ylabel('Value')
plt.title('Adaptation of Q and R over Time')
plt.legend()
plt.grid()
plt.show()